# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonets,60.9847,32.9698,12.03,87,4,2.00,RU,1720575089
1,1,bethel,41.3712,-73.4140,26.62,78,97,1.46,US,1720575090
2,2,gadzhiyevo,69.2551,33.3362,9.79,79,100,3.09,RU,1720575092
3,3,ribeira grande,38.5167,-28.7000,22.65,87,67,0.45,PT,1720575093
4,4,tura,25.5198,90.2201,25.39,96,100,3.52,IN,1720575095


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
    
)

# Display the map plot
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values

perfect_temp_df = city_data_df.loc[((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)), :]
easy_breeze_df = perfect_temp_df.loc[(perfect_temp_df["Wind Speed"] < 4.5), :]
clear_skies_df = easy_breeze_df.loc[(easy_breeze_df["Cloudiness"] == 0), :]
# Drop any rows with null values
ideal_conditions_df = clear_skies_df.dropna()

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,khanty-mansiysk,61.0042,69.0019,22.06,78,0,0.00,RU,1720575158
106,106,cheremukhovo,60.3484,59.9851,21.60,63,0,0.71,RU,1720575233
115,115,moose jaw,50.4001,-105.5344,26.14,53,0,3.09,CA,1720575158
119,119,torremaggiore,41.6907,15.2963,23.79,69,0,0.90,IT,1720575251
272,272,zadar,44.1197,15.2422,23.34,94,0,0.51,HR,1720575471
308,308,puerto real,36.5282,-6.1901,21.09,88,0,0.00,ES,1720575522
321,321,edmundston,47.3737,-68.3251,21.74,85,0,3.09,CA,1720575541
333,333,sault ste. marie,46.5168,-84.3333,21.31,78,0,2.57,CA,1720575557
334,334,westport,41.1415,-73.3579,26.90,81,0,2.57,US,1720575550
338,338,sar-e pul,35.8333,66.1667,21.81,29,0,0.81,AF,1720575564


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,khanty-mansiysk,RU,61.0042,69.0019,78,
106,cheremukhovo,RU,60.3484,59.9851,63,
115,moose jaw,CA,50.4001,-105.5344,53,
119,torremaggiore,IT,41.6907,15.2963,69,
272,zadar,HR,44.1197,15.2422,94,
308,puerto real,ES,36.5282,-6.1901,88,
321,edmundston,CA,47.3737,-68.3251,85,
333,sault ste. marie,CA,46.5168,-84.3333,78,
334,westport,US,41.1415,-73.3579,81,
338,sar-e pul,AF,35.8333,66.1667,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

limit = 1
params = {"categories":categories,
          "limit" : limit,
          "apiKey" : geoapify_key
}
# categories, filter, bias, limit, apikey
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_latitude = row["Lat"]
    city_longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_longitude},{city_latitude},{radius}"
    params["bias"] = f"proximity:{city_longitude},{city_latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # name_address = f"{base_url}?categories={categories}&filter={filter}&bias={bias}&limit={limit}&apiKey={geoapify_key}"
    
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format

    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
khanty-mansiysk - nearest hotel: Гамма
cheremukhovo - nearest hotel: Метла
moose jaw - nearest hotel: Temple Gardens Mineral Spa Resort
torremaggiore - nearest hotel: Albergo Fontana
zadar - nearest hotel: No hotel found
puerto real - nearest hotel: Hostal Bahía de Cádiz
edmundston - nearest hotel: Comfort Inn
sault ste. marie - nearest hotel: Holiday Inn
westport - nearest hotel: Norwalk Inn and Conference Center
sar-e pul - nearest hotel: No hotel found
zhanibek - nearest hotel: No hotel found
noci - nearest hotel: Abate Masseria & Resort
gargalianoi - nearest hotel: Messinian Horizons
belmonte - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
sirte - nearest hotel: فندق المدينة-سرت
forio - nearest hotel: Hotel Terme Colella


,City,Country,Lat,Lng,Humidity,Hotel Name
49,khanty-mansiysk,RU,61.0042,69.0019,78,Гамма
106,cheremukhovo,RU,60.3484,59.9851,63,Метла
115,moose jaw,CA,50.4001,-105.5344,53,Temple Gardens Mineral Spa Resort
119,torremaggiore,IT,41.6907,15.2963,69,Albergo Fontana
272,zadar,HR,44.1197,15.2422,94,No hotel found
308,puerto real,ES,36.5282,-6.1901,88,Hostal Bahía de Cádiz
321,edmundston,CA,47.3737,-68.3251,85,Comfort Inn
333,sault ste. marie,CA,46.5168,-84.3333,78,Holiday Inn
334,westport,US,41.1415,-73.3579,81,Norwalk Inn and Conference Center
338,sar-e pul,AF,35.8333,66.1667,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)